# 7.8 연습




리스트7.28 데이터셋 읽어 들이기와 전처리

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

batch_size = 32
epochs = 20
n_class = 10

x_train = x_train / 255  # 0부터 1의 범위에 넣는다
x_test = x_test / 255
t_train = tf.keras.utils.to_categorical(t_train, n_class)  # one-hot 표현으로
t_test = tf.keras.utils.to_categorical(t_test, n_class)

170508288/170498071 [==============================] - 14s 0us/step


리스트7.29 모델의 구축

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))  # 제로 패딩, 배치 크기 이외의 이미지의 형태를 지정 
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 1차원 배열로 변환
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))  # 드롭 아웃
model.add(Dense(n_class))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 64)        18496     
                                                        

리스트7.30 학습 

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

generator = ImageDataGenerator(
           rotation_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)

history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


1563/1563 [==============================] - 37s 17ms/step - loss: 1.4739 - accuracy: 0.4641 - val_loss: 1.1226 - val_accuracy: 0.6019
Epoch 2/20
1563/1563 [==============================] - 27s 17ms/step - loss: 1.0670 - accuracy: 0.6248 - val_loss: 0.8933 - val_accuracy: 0.6834
Epoch 3/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.9127 - accuracy: 0.6827 - val_loss: 0.7830 - val_accuracy: 0.7294
Epoch 4/20
1563/1563 [==============================] - 27s 17ms/step - loss: 0.8241 - accuracy: 0.7139 - val_loss: 0.7848 - val_accuracy: 0.7295
Epoch 5/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.7628 - accuracy: 0.7351 - val_loss: 0.7038 - val_accuracy: 0.7546
Epoch 6/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.7074 - accuracy: 0.7546 - val_loss: 0.7078 - val_accuracy: 0.7588
Epoch 7/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.6738 - accuracy: 0.7659 - val_loss: 0.6706 - val_accura

리스트7.31 학습의 추이

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']  # 훈련용 데이터의 오차
train_acc = history.history['accuracy']  # 훈련용 데이터의 정밀도
val_loss = history.history['val_loss']  # 검증용 데이터의 오차
val_acc = history.history['val_accuracy']  # 검증용 데이터의 정밀도

plt.plot(np.arange(len(train_loss)), train_loss, label='loss')
plt.plot(np.arange(len(val_loss)), val_loss, label='val_loss')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label='acc')
plt.plot(np.arange(len(val_acc)), val_acc, label='val_acc')
plt.legend()
plt.show()

리스트7.32 모델의 평가

In [ ]:
loss, accuracy = model.evaluate(x_test, t_test)
print("오차: ", loss, "정밀도: ", accuracy)